In [ ]:
!pip install torch fasttext python-docx PyPDF2 beautifulsoup4 reportlab bitsandbytes -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 745.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s

In [ ]:
!pip uninstall -y numpy transformers
!pip install numpy==1.26.4
!pip install transformers --no-cache-dir

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
peft 0.14.0 requires transformers, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 217.8 MB/s eta 0:00:00


After installing the necessary libraries, it's important to restart the session. This ensures that the newly installed packages are properly loaded into the environment. Without restarting, the current session may not recognize the changes, and the libraries might not work as expected.



In [ ]:
import os
import re
import torch
import traceback
from pathlib import Path
from typing import Dict, List, Optional, Union, Any

### **Decorator to measure tokens**

In [ ]:
import time
import functools
from typing import Callable, Any, Optional

def measure_token_processing(process_name: Optional[str] = None):
    """Decorator to measure token processing speed across different tasks."""

    def decorator(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Determine the process name
            task_name = process_name or func.__name__

            # Get the instance (self) from args
            instance = args[0] if args else None

            # Find text to tokenize - could be in different places depending on function
            text = None
            # Check args - likely the second argument for most methods
            if len(args) > 1 and isinstance(args[1], str):
                text = args[1]
            elif len(args) > 1 and isinstance(args[1], list) and all(isinstance(t, str) for t in args[1]):
                text = ' '.join(args[1])  # Join list of strings

            # Check kwargs for text
            elif 'text' in kwargs and isinstance(kwargs['text'], str):
                text = kwargs['text']
            elif 'texts' in kwargs and isinstance(kwargs['texts'], list):
                text = ' '.join(kwargs['texts'])
            elif 'prompt' in kwargs and isinstance(kwargs['prompt'], str):
                text = kwargs['prompt']

            # Find appropriate tokenizer
            tokenizer = None
            if instance and hasattr(instance, 'tokenizer'):
                tokenizer = instance.tokenizer
            elif instance and hasattr(instance, 'embedding_model') and hasattr(instance.embedding_model, 'tokenizer'):
                tokenizer = instance.embedding_model.tokenizer

            # Count tokens if we have both text and tokenizer
            input_tokens = 0
            if text and tokenizer:
                try:
                    if hasattr(tokenizer, 'encode'):
                        input_tokens = len(tokenizer.encode(text))
                    elif hasattr(tokenizer, '__call__'):
                        input_tokens = len(tokenizer(text)['input_ids'])
                except:
                    # Fallback to rough estimate (approx 4 chars per token)
                    input_tokens = len(text) // 4
            elif text:
                # Very rough approximation if no tokenizer available
                input_tokens = len(text.split())

            # Start timing
            start_time = time.perf_counter()

            # Execute the function
            result = func(*args, **kwargs)

            # End timing
            end_time = time.perf_counter()

            # Calculate tokens per second
            processing_time = end_time - start_time
            tokens_per_second = input_tokens / processing_time if processing_time > 0 else 0

            # Output measurements
            print(f"⏱️ {task_name}: {input_tokens} tokens processed in {processing_time:.2f}s ({tokens_per_second:.2f} tokens/sec)\n")

            # If result is a string, we could measure output tokens too
            if isinstance(result, str) and tokenizer:
                try:
                    output_tokens = len(tokenizer.encode(result))
                    total_tokens = input_tokens + output_tokens
                    throughput = total_tokens / processing_time if processing_time > 0 else 0
                    print(f"   Total (in+out): {total_tokens} tokens at {throughput:.2f} tokens/sec\n\n")
                except:
                    pass

            return result
        return wrapper
    return decorator

### **Translation Service**

A simple yet effective document translation system that uses local LLMs:

- **Local Language Model Support**: Uses lightweight models like NLLB and M2M100 for translation
- **Format Preservation**: Maintains document structure, paragraphs, lists, and tables during translation
- **Chunking Strategy**: Intelligently splits text into manageable pieces while preserving semantic integrity
- **Multi-Document Support**: Handles various formats including PDF, DOCX, and plain text
- **Language Auto-Detection**: Automatically identifies source language when not specified
- **Resource Optimization**: Implements model caching and fallback to smaller models when needed
- **Table and Code Handling**: Preserves special content formats that shouldn't be fully translated

This service provides efficient translation capabilities that can run entirely on local hardware without requiring external API calls or cloud services.

In [ ]:
# Login to HF to access LLMs
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
class TranslationService:
    """
    A service for translating documents between languages using local LLMs.
    Focuses on maintaining document structure and formatting during translation.
    """
    # Class-level cache for loaded models
    _loaded_translation_model = None
    _loaded_translation_model_name = None

    def __init__(
        self,
        model_name: str = "facebook/nllb-200-distilled-600M",  # Smaller NLLB model suitable for local use
        device: str = "auto",
        preserve_formatting: bool = True
    ):

        self.model_name = model_name
        self.preserve_formatting = preserve_formatting

        # Determine device
        if device == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        print(f"-----Initializing translation service with model {model_name} on {self.device}-----\n")

        # Load translation model (with caching)
        self._load_model()

        # Map of language codes for NLLB
        self.language_code_map = self._initialize_language_codes()

    def _load_model(self):
        """Load the translation model and tokenizer, with caching."""

        try:
            # Check if the model is already loaded
            if (TranslationService._loaded_translation_model is not None and
                TranslationService._loaded_translation_model_name == self.model_name):
                print("Translation model already loaded. Reusing from cache.")
                self.model = TranslationService._loaded_translation_model
                self.tokenizer = self.model.tokenizer
                return

            # Import necessary libraries
            from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

            print(f"-----Loading translation model: {self.model_name}-----\n")

            # Load the tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

            # Load model with appropriate settings based on device
            if self.device == "cuda":
                self.model = AutoModelForSeq2SeqLM.from_pretrained(
                    self.model_name,
                    device_map="auto",
                    torch_dtype=torch.float16,
                    load_in_8bit=True
                )
            else:
                self.model = AutoModelForSeq2SeqLM.from_pretrained(
                    self.model_name,
                    device_map={"": self.device}
                )

            # Cache the loaded model
            TranslationService._loaded_translation_model = self.model
            TranslationService._loaded_translation_model_name = self.model_name

        except Exception as e:
            print(f"Error loading translation model: {str(e)}")
            print("-----Attempting to load a smaller fallback model...-----\n")

            try:
                # Fallback to an even smaller model
                fallback_model = "facebook/m2m100_418M"
                self.model_name = fallback_model

                from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

                self.tokenizer = M2M100Tokenizer.from_pretrained(fallback_model)
                self.model = M2M100ForConditionalGeneration.from_pretrained(
                    fallback_model,
                    device_map={"": self.device}
                )

                # Cache the fallback model
                TranslationService._loaded_translation_model = self.model
                TranslationService._loaded_translation_model_name = fallback_model

            except Exception as e2:
                print(f"Error loading fallback model: {str(e2)}")
                print("-----Translation service will not be functional.-----\n")
                self.model = None
                self.tokenizer = None

    def _initialize_language_codes(self):
        """Initialize the mapping of language names to NLLB language codes."""

        # This is a partial list of NLLB language codes - expand as needed
        return {
            "english": "eng_Latn",
            "arabic": "arb_Arab",
            "french": "fra_Latn",
            "spanish": "spa_Latn",
            "german": "deu_Latn",
            "chinese": "zho_Hans",
            "japanese": "jpn_Jpan",
            "russian": "rus_Cyrl",
            "portuguese": "por_Latn",
            "italian": "ita_Latn",
            "dutch": "nld_Latn",
            "swedish": "swe_Latn",
            "korean": "kor_Hang",
            "turkish": "tur_Latn",
            "hindi": "hin_Deva",
            "bengali": "ben_Beng",
            "urdu": "urd_Arab",
            "persian": "fas_Arab",
            "thai": "tha_Thai",
            "vietnamese": "vie_Latn",
            "indonesian": "ind_Latn",
            "malay": "zsm_Latn",
            "polish": "pol_Latn",
            "ukrainian": "ukr_Cyrl",
            "greek": "ell_Grek",
            "czech": "ces_Latn",
            "finnish": "fin_Latn",
            "hungarian": "hun_Latn"
        }

    def get_language_code(self, language: str) -> str:
        """Get the NLLB language code for a given language name."""

        language = language.lower().strip()
        if language in self.language_code_map:
            return self.language_code_map[language]
        else:
            # If language not found, attempt to find partial matches
            for lang, code in self.language_code_map.items():
                if language in lang or lang in language:
                    return code

            # Default to English if no match found
            print(f"Warning: Language '{language}' not found in mapping. Using English as default.\n")
            return self.language_code_map["english"]

    def detect_language(self, text: str) -> str:
        """Detect the language of the text."""
        try:
            # Import fasttext for language detection
            import fasttext

            # Check if the model exists and download if needed
            model_path = "lid.176.bin"
            if not os.path.exists(model_path):
                print("-----Downloading fastText language detection model...-----\n")
                import urllib.request
                url = "https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin"
                urllib.request.urlretrieve(url, model_path)

            # Load the model
            model = fasttext.load_model(model_path)

            # Detect language
            prediction = model.predict(text.replace("\n", " "))

            # Extract language code and map to NLLB code
            detected_lang = prediction[0][0].replace("__label__", "")

            # Map ISO code to our language name
            iso_to_name = {
                "en": "english",
                "ar": "arabic",
                "fr": "french",
                "es": "spanish",
                "de": "german",
                "zh": "chinese",
                "ja": "japanese",
                "ru": "russian",
                "pt": "portuguese",
                "it": "italian",
                "nl": "dutch",
                "sv": "swedish",
                "ko": "korean",
                "tr": "turkish",
                "hi": "hindi",
                "bn": "bengali",
                "ur": "urdu",
                "fa": "persian",
                "th": "thai",
                "vi": "vietnamese",
                "id": "indonesian",
                "ms": "malay",
                "pl": "polish",
                "uk": "ukrainian",
                "el": "greek",
                "cs": "czech",
                "fi": "finnish",
                "hu": "hungarian"
            }

            if detected_lang in iso_to_name:
                return self.get_language_code(iso_to_name[detected_lang])
            else:
                print(f"Warning: Detected language '{detected_lang}' not mapped to NLLB code. Using English.\n")
                return self.language_code_map["english"]

        except Exception as e:
            print(f"Error in language detection: {str(e)}")
            print("Falling back to English")
            return self.language_code_map["english"]

    def _split_into_chunks(self, text: str, max_length: int = 512) -> list:
        """Split text into chunks for translation, preserving structure."""

        # If text is shorter than max_length, return it as a single chunk
        if len(self.tokenizer.encode(text)) <= max_length:
            return [text]

        # First, split by paragraphs
        paragraphs = text.split("\n\n")
        chunks = []
        current_chunk = ""

        for paragraph in paragraphs:
            # If paragraph is too long, split it by sentences
            if len(self.tokenizer.encode(paragraph)) > max_length:
                sentences = self._split_into_sentences(paragraph)
                for sentence in sentences:
                    # If adding this sentence would make the chunk too long, start a new chunk
                    if len(self.tokenizer.encode(current_chunk + sentence)) > max_length and current_chunk:
                        chunks.append(current_chunk)
                        current_chunk = sentence
                    else:
                        current_chunk += sentence
            else:
                # If adding this paragraph would make the chunk too long, start a new chunk
                if len(self.tokenizer.encode(current_chunk + paragraph + "\n\n")) > max_length and current_chunk:
                    chunks.append(current_chunk)
                    current_chunk = paragraph + "\n\n"
                else:
                    current_chunk += paragraph + "\n\n"

        # Add the last chunk if it's not empty
        if current_chunk:
            chunks.append(current_chunk)

        return chunks

    def _split_into_sentences(self, text: str) -> list:
        """Split text into sentences."""

        # Simple sentence splitting - enhance this for better results
        import re
        sentence_endings = r'(?<=[.!?])\s+'
        sentences = re.split(sentence_endings, text)
        return [s + ". " if not s.endswith(('.', '!', '?')) else s + " " for s in sentences if s]

    def _preserve_formatting(self, original: str, translated: str) -> str:
        """Attempt to preserve formatting from original text in the translated text."""

        if not self.preserve_formatting:
            return translated

        # Preserve paragraph breaks
        if "\n\n" in original and "\n\n" not in translated:
            # Count paragraphs in original
            orig_paragraphs = original.split("\n\n")
            if len(orig_paragraphs) > 1:
                # Try to split translated text into similar number of paragraphs
                trans_sentences = self._split_into_sentences(translated)
                sentences_per_paragraph = max(1, len(trans_sentences) // len(orig_paragraphs))

                new_translated = ""
                for i in range(0, len(trans_sentences), sentences_per_paragraph):
                    paragraph = "".join(trans_sentences[i:i+sentences_per_paragraph])
                    new_translated += paragraph.strip() + "\n\n"

                translated = new_translated.strip()

        # Preserve bullet points and numbering
        bullet_pattern = r'^\s*[-•*]\s+'
        number_pattern = r'^\s*\d+[.)]?\s+'

        if re.search(bullet_pattern, original, re.MULTILINE) and not re.search(bullet_pattern, translated, re.MULTILINE):
            # Original has bullet points but translated doesn't
            orig_lines = original.split('\n')
            trans_lines = translated.split('\n')

            if len(orig_lines) == len(trans_lines):
                # Match line by line
                for i in range(len(orig_lines)):
                    if re.match(bullet_pattern, orig_lines[i]):
                        bullet = re.match(bullet_pattern, orig_lines[i]).group(0)
                        trans_lines[i] = bullet + trans_lines[i].lstrip()

                translated = '\n'.join(trans_lines)

        # Preserve tables (simple implementation)
        if '|' in original and '|' not in translated:
            # Try to identify table structures and preserve them
            table_rows = re.findall(r'^\s*\|.*\|\s*$', original, re.MULTILINE)
            if table_rows:
                # Extract table content and structure
                table_content = {}
                for i, row in enumerate(table_rows):
                    cells = [cell.strip() for cell in row.split('|')[1:-1]]
                    table_content[i] = cells

                # Translate table content while preserving structure
                translated_table = {}
                for row_idx, cells in table_content.items():
                    translated_cells = []
                    for cell in cells:
                        if cell and not all(c in '- :' for c in cell):  # Skip separator rows
                            translated_cell = self.translate(cell, target_language="current")
                            translated_cells.append(translated_cell)
                        else:
                            translated_cells.append(cell)
                    translated_table[row_idx] = translated_cells

                # Replace original table with translated one
                for i, row in enumerate(table_rows):
                    if i in translated_table:
                        new_row = '| ' + ' | '.join(translated_table[i]) + ' |'
                        translated = translated.replace(row, new_row)

        # Preserve code blocks
        code_block_pattern = r'```(?:\w+)?\n[\s\S]*?\n```'
        code_blocks = re.findall(code_block_pattern, original)
        if code_blocks:
            for code_block in code_blocks:
                # Don't translate code blocks, keep them as is
                if code_block in original and code_block not in translated:
                    translated = translated + "\n\n" + code_block

        return translated

    @measure_token_processing("Translation")
    def translate(self, text: str, target_language: str, source_language: str = None) -> str:
        """Translate text to the target language."""

        if not text.strip():
            return text

        if self.model is None:
            return "Translation model not loaded properly."

        try:
            # Special case for "current" language target - just return original text
            if target_language.lower() == "current":
                return text

            # Get language codes
            target_lang_code = self.get_language_code(target_language)

            # Auto-detect source language if not provided
            if not source_language:
                source_lang_code = self.detect_language(text)
                print(f"-----Detected source language code: {source_lang_code}-----\n")
            else:
                source_lang_code = self.get_language_code(source_language)

            # If source and target languages are the same, return original text
            if source_lang_code == target_lang_code:
                return text

            # Split text into manageable chunks
            chunks = self._split_into_chunks(text)
            translated_chunks = []

            # Translate each chunk
            for chunk in chunks:
                if not chunk.strip():
                    translated_chunks.append(chunk)
                    continue

                # Prepare inputs for the model
                if "nllb" in self.model_name.lower():
                    # NLLB model
                    self.tokenizer.src_lang = source_lang_code
                    inputs = self.tokenizer(chunk, return_tensors="pt")
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        generated_tokens = self.model.generate(
                            **inputs,
                            forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(target_lang_code),
                            max_length=1024,
                            num_beams=5,
                            length_penalty=1.0
                        )

                    translated_chunk = self.tokenizer.batch_decode(
                        generated_tokens, skip_special_tokens=True
                    )[0]

                elif "m2m100" in self.model_name.lower():
                    # M2M100 model
                    self.tokenizer.src_lang = source_lang_code.split('_')[0]
                    inputs = self.tokenizer(chunk, return_tensors="pt")
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    self.tokenizer.tgt_lang = target_lang_code.split('_')[0]
                    with torch.no_grad():
                        generated_tokens = self.model.generate(
                            **inputs,
                            forced_bos_token_id=self.tokenizer.get_lang_id(target_lang_code.split('_')[0]),
                            max_length=1024,
                            num_beams=5,
                            length_penalty=1.0
                        )

                    translated_chunk = self.tokenizer.batch_decode(
                        generated_tokens, skip_special_tokens=True
                    )[0]

                else:
                    # Generic approach for other models
                    inputs = self.tokenizer(chunk, return_tensors="pt").to(self.device)

                    with torch.no_grad():
                        outputs = self.model.generate(
                            **inputs,
                            max_length=1024,
                            num_beams=5,
                            length_penalty=1.0
                        )

                    translated_chunk = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

                # Preserve formatting
                if self.preserve_formatting:
                    translated_chunk = self._preserve_formatting(chunk, translated_chunk)

                translated_chunks.append(translated_chunk)

            # Combine translated chunks
            full_translation = " ".join(translated_chunks)

            # Clean up any artifacts from the chunking process
            full_translation = full_translation.replace("  ", " ")
            full_translation = re.sub(r'\s+\.', '.', full_translation)
            full_translation = re.sub(r'\s+,', ',', full_translation)

            return full_translation

        except Exception as e:
            print(f"Error in translation: {str(e)}")
            traceback.print_exc()
            return f"Translation error: {str(e)}"

    def translate_file(self, file_path: str, target_language: str, source_language: str = None) -> str:
        """Translate a file to the target language."""

        try:
            # Determine file type and handle accordingly
            file_path = Path(file_path)
            file_extension = file_path.suffix.lower()

            # Create output file path
            output_dir = file_path.parent / "translated"
            os.makedirs(output_dir, exist_ok=True)

            output_file = output_dir / f"{file_path.stem}_{target_language}{file_extension}"

            if file_extension in ['.docx', '.doc']:
                # Word document - use python-docx
                return self._translate_docx(file_path, output_file, target_language, source_language)

            elif file_extension == '.pdf':
                # PDF - extract text, translate, create new PDF
                return self._translate_pdf(file_path, output_file, target_language, source_language)

            else:
                # Default to treating as plain text
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                translated_content = self.translate(content, target_language, source_language)

            # Write translated content to output file
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(translated_content)

            return str(output_file)

        except Exception as e:
            print(f"Error translating file: {str(e)}")
            traceback.print_exc()
            return f"File translation error: {str(e)}"

    def _translate_json(self, data, target_language: str, source_language: str = None):
        """Recursively translate values in a JSON object."""

        if isinstance(data, str):
            return self.translate(data, target_language, source_language)
        elif isinstance(data, list):
            return [self._translate_json(item, target_language, source_language) for item in data]
        elif isinstance(data, dict):
            return {k: self._translate_json(v, target_language, source_language) for k, v in data.items()}
        else:
            return data


    def _translate_docx(self, input_path, output_path, target_language: str, source_language: str = None) -> str:
        """Translate a Word document."""

        try:
            import docx

            # Load the document
            doc = docx.Document(input_path)

            # Translate each paragraph
            for para in doc.paragraphs:
                if para.text.strip():
                    translated_text = self.translate(para.text, target_language, source_language)

                    # Preserve runs with their formatting
                    if len(para.runs) > 1:
                        # This is an approximation and won't be perfect
                        words = translated_text.split()
                        runs_text = [run.text for run in para.runs if run.text.strip()]

                        # Calculate distribution
                        total_original_length = sum(len(text) for text in runs_text)
                        words_per_run = []

                        for run_text in runs_text:
                            proportion = len(run_text) / total_original_length if total_original_length > 0 else 0
                            words_for_run = max(1, int(proportion * len(words)))
                            words_per_run.append(words_for_run)

                        # Adjust to make sure all words are distributed
                        while sum(words_per_run) < len(words):
                            words_per_run[-1] += 1

                        # Apply to runs
                        word_index = 0
                        for i, run in enumerate(para.runs):
                            if i < len(words_per_run):
                                run_words = words[word_index:word_index + words_per_run[i]]
                                run.text = " ".join(run_words)
                                word_index += words_per_run[i]
                            else:
                                run.text = ""
                    else:
                        # Simple case - just replace text
                        para.text = translated_text

            # Translate table contents
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        for para in cell.paragraphs:
                            if para.text.strip():
                                para.text = self.translate(para.text, target_language, source_language)

            # Save the document
            doc.save(output_path)

            return str(output_path)

        except Exception as e:
            print(f"Error translating Word document: {str(e)}")
            return f"Word document translation error: {str(e)}"


    def _translate_pdf(self, input_path, output_path, target_language: str, source_language: str = None) -> str:
        """
        Extract text from PDF, translate it page by page, and save only as text files.
        Returns the path to the complete translated text file.
        """
        try:
            import PyPDF2
            import os
            from pathlib import Path

            # Convert Path objects to strings
            input_path_str = str(input_path)

            # Create output directory and path for text file
            output_dir = os.path.dirname(str(output_path))
            os.makedirs(output_dir, exist_ok=True)

            # Create text file path (replace .pdf with .txt if needed)
            output_txt = str(output_path).replace('.pdf', '.txt')

            # Create directory for individual pages
            base_filename = os.path.basename(output_txt).split('.')[0]
            pages_dir = os.path.join(output_dir, f"{base_filename}_pages")
            os.makedirs(pages_dir, exist_ok=True)

            print(f"Starting page-by-page translation of PDF: {input_path_str}")

            # Open the PDF
            with open(input_path_str, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                total_pages = len(reader.pages)
                print(f"PDF has {total_pages} pages")

                all_translated_text = ""

                # Process each page individually
                for page_num in range(total_pages):
                    try:
                        print(f"Processing page {page_num + 1} of {total_pages}")

                        # Extract text from this page
                        page_text = reader.pages[page_num].extract_text()

                        if not page_text.strip():
                            print(f"Page {page_num + 1} appears to be empty or contains only images")
                            page_info = f"\n\n--- Page {page_num + 1} (Empty or contains only images) ---\n\n"
                            all_translated_text += page_info

                            # Save empty page file as a placeholder
                            page_file = os.path.join(pages_dir, f"page_{page_num + 1}.txt")
                            with open(page_file, 'w', encoding='utf-8') as f:
                                f.write(page_info)
                            continue

                        # Translate this page's text
                        translated_page = self.translate(page_text, target_language, source_language)

                        # Add page information
                        page_content = f"\n\n--- Page {page_num + 1} ---\n\n{translated_page}"
                        all_translated_text += page_content

                        # Save individual page translation
                        page_file = os.path.join(pages_dir, f"page_{page_num + 1}.txt")
                        with open(page_file, 'w', encoding='utf-8') as f:
                            f.write(page_content)

                        print(f"Saved translated page {page_num + 1} to {page_file}")

                    except Exception as e:
                        print(f"Error processing page {page_num + 1}: {str(e)}")
                        error_message = f"\n\n--- Error on Page {page_num + 1}: {str(e)} ---\n\n"
                        all_translated_text += error_message

                        # Save error information
                        page_file = os.path.join(pages_dir, f"page_{page_num + 1}_error.txt")
                        with open(page_file, 'w', encoding='utf-8') as f:
                            f.write(error_message)

            # Save the complete text version
            with open(output_txt, 'w', encoding='utf-8') as f:
                f.write(all_translated_text)
            print(f"Saved complete translated text to {output_txt}")
            print(f"Individual translated pages saved to {pages_dir}")

            return output_txt

        except Exception as e:
            print(f"Error translating PDF: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"PDF translation error: {str(e)}"

In [ ]:
translator = TranslationService(
    model_name = "facebook/nllb-200-distilled-600M",
    device = "auto",
    preserve_formatting = True
)

-----Initializing translation service with model facebook/nllb-200-distilled-600M on cuda-----

-----Loading translation model: facebook/nllb-200-distilled-600M-----



tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

**English to French**

In [ ]:
# Let's try translating a simple sentence from English to French
text = "The quick brown fox jumps over the lazy dog."
source_language = "en"
target_language = "fr"

translated_text = translator.translate(text, target_language, source_language)
print("Translated text:", translated_text)

⏱️ Translation: 16 tokens processed in 2.19s (7.30 tokens/sec)

   Total (in+out): 33 tokens at 15.05 tokens/sec


Translated text: Le renard brun rapide saute sur le chien paresseux.


**English to Arabic**

In [ ]:
# Let's try translating to Arabic
text = "The quick brown fox jumps over the lazy dog."
source_language = "en"
target_language = "ar"

translated_text = translator.translate(text, target_language, source_language)
print("Translated text:", translated_text)

⏱️ Translation: 16 tokens processed in 1.04s (15.42 tokens/sec)

   Total (in+out): 35 tokens at 33.73 tokens/sec


Translated text: الثعلب البني السريع يقفز فوق الكلب الكسول.


**English to Russian**

In [ ]:
# Let's try translating to Russian
text = "The quick brown fox jumps over the lazy dog."
source_language = "en"
target_language = "ru"

translated_text = translator.translate(text, target_language, source_language)
print("Translated text:", translated_text)

⏱️ Translation: 16 tokens processed in 2.28s (7.03 tokens/sec)

   Total (in+out): 35 tokens at 15.38 tokens/sec


Translated text: Быстрая коричневая лиса прыгает над ленивой собакой.


**Instead of directly translating entire files, let's perform the translation on the chunks created during the chunking process**

In [ ]:
# Unzipping chunked data
!unzip /content/chunked_data.zip

Archive:  /content/chunked_data.zip
   creating: content/chunked_data/
  inflating: content/chunked_data/The-Alchemist_chunks.json  
  inflating: content/chunked_data/Ocean_ecogeochemistry_A_review_chunks.json  
  inflating: content/chunked_data/Stats_chunks.json  
  inflating: content/chunked_data/new-approaches-and-procedures-for-cancer-treatment_chunks.json  
  inflating: content/chunked_data/all_chunks.json  
  inflating: content/chunked_data/The_Plan_of_the_Giza_Pyramids_chunks.json  
  inflating: content/chunked_data/M.Sc. Applied Psychology_chunks.json  
  inflating: content/chunked_data/Dataset summaries and citations_chunks.json  


In [ ]:
!mkdir chunked_data
!mv /content/content/chunked_data/* /content/chunked_data
!rm -rf /content/content/

### Loading and Grouping Document Chunks by Source

In [ ]:
import json

chunks_dir = "/content/chunked_data"
def load_chunks(filename: str = "all_chunks.json") -> List[Dict[str, Any]]:
    """Load document chunks from a JSON file."""
    filepath = os.path.join(chunks_dir, filename)
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            chunks = json.load(f)
        print(f"Loaded {len(chunks)} chunks from {filepath}")
        return chunks
    except Exception as e:
        print(f"Error loading chunks from {filepath}: {str(e)}")
        return []

In [ ]:
def group_chunks_by_source(chunks: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """Group chunks by their source document."""
    grouped = {}
    for chunk in chunks:
        source = chunk.get("source", "unknown")
        if source not in grouped:
            grouped[source] = []
        grouped[source].append(chunk)

    # Sort chunks by chunk number within each source
    for source in grouped:
        grouped[source].sort(key=lambda x: x.get("chunk_number", 0))

    return grouped

In [17]:
# Group chunks by source
chunks = load_chunks()
grouped_chunks = group_chunks_by_source(chunks)

Loaded 599 chunks from /content/chunked_data/all_chunks.json


In [18]:
grouped_chunks.keys()

dict_keys(['new-approaches-and-procedures-for-cancer-treatment.pdf', 'The_Plan_of_the_Giza_Pyramids.pdf', 'The-Alchemist.pdf', 'Stats.docx', 'Ocean_ecogeochemistry_A_review.pdf', 'Dataset summaries and citations.docx', 'M.Sc. Applied Psychology.docx'])

Now let's try translation from 'Stats.docx' document

In [19]:
grouped_chunks['Stats.docx'][0]['text']

'Introduction The purpose of the Machine Readable file (“the file”) is to provide regression coefficients and intercepts for different components to calculate low, mid, and high (10th, 50th, and 90th percentile) material price estimates and labor multipliers/add-ons to estimate new construction and retrofit project costs. The file provides a list of envelope and non-envelope components (e.g., Windows, Water Heaters) and any associated classes within those components (e.g., Low Emissivity, Electric Instantaneous).'

In [ ]:
# Translation to Arabic
translation = translator.translate(grouped_chunks['Stats.docx'][0]['text'], target_language="ar")

-----Downloading fastText language detection model...-----

-----Detected source language code: eng_Latn-----

⏱️ Translation: 134 tokens processed in 8.98s (14.93 tokens/sec)
   Total (in+out): 282 tokens at 31.41 tokens/sec


In [ ]:
translation

'مقدمة الغرض من ملف القراءة الآلية (the file) هو توفير معايير التراجع والقاطع للمكونات المختلفة لحساب تقديرات أسعار المواد المنخفضة والمتوسطة والعالية (10 و50 و90 مئوية) ومضاعفات العمالة / إضافات لتقدير تكاليف المشروعات الجديدة في البناء والتعديل. يقدم الملف قائمة بالمكونات الغلافية وغير الغلافية (مثل Windows و Heaters Water) وأي فئات مرتبطة ضمن تلك المكونات (مثل Low Emissivity و Electric Instantaneous).'

In [ ]:
# Translation to French
translation = translator.translate(grouped_chunks['Stats.docx'][0]['text'], target_language="fr")

-----Detected source language code: eng_Latn-----

⏱️ Translation: 134 tokens processed in 10.07s (13.31 tokens/sec)
   Total (in+out): 236 tokens at 23.44 tokens/sec


In [ ]:
translation

"Introduction Le but du fichier Machine Readable (the file) est de fournir des coefficients de régression et des interceptions pour différents composants pour calculer les estimations basses, moyennes et élevées (10e, 50e et 90e percentiles) des prix des matériaux et des multiplicateurs de main-d'œuvre / add-ons pour estimer les coûts de nouveaux projets de construction et de rénovation."

In [ ]:
# Translation to Japanese
translation = translator.translate(grouped_chunks['Stats.docx'][0]['text'], target_language="ja")

-----Detected source language code: eng_Latn-----

⏱️ Translation: 134 tokens processed in 7.91s (16.95 tokens/sec)
   Total (in+out): 267 tokens at 33.77 tokens/sec


In [ ]:
translation

'導入 マシンリーダブルファイル (the file) の目的は,低,中,高 (10 番目の,50 番目の,90 番目の百分点) の材料価格推定と労働倍数/アドオンを計算するために,異なるコンポーネントの回帰系数と傍受を提供することです.このファイルは,新しい建設と改装プロジェクトのコストを推定するために,包装および非包装コンポーネント (例えば,Windows,Water Heaters) とそれらのコンポーネント内の関連クラス (例えば,Low Emissivity, Electric Instantaneous) のリストを提供します.'



> We can translate text into a wide range of languages using the current model. The supported languages are defined in a dictionary **`iso_to_name`** that maps language names to their corresponding NLLB language codes. This dictionary is easily extendable, allowing us to add support for even more languages as needed.



**Now let's try to tranlsate few chunks of 'The Plan of the Giza Pyramids' document but in json**

In [21]:
translated_jsons = []
for chunk in grouped_chunks['The_Plan_of_the_Giza_Pyramids.pdf'][:5]: # First 5 chunks
    translated_json_data = translator._translate_json(chunk, target_language="ar", source_language="en")
    translated_jsons.append(translated_json_data)

⏱️ Translation: 19 tokens processed in 1.05s (18.02 tokens/sec)

   Total (in+out): 32 tokens at 30.36 tokens/sec


⏱️ Translation: 12 tokens processed in 1.15s (10.46 tokens/sec)

   Total (in+out): 22 tokens at 19.17 tokens/sec


⏱️ Translation: 4 tokens processed in 0.64s (6.21 tokens/sec)

   Total (in+out): 8 tokens at 12.42 tokens/sec


⏱️ Translation: 7 tokens processed in 0.83s (8.47 tokens/sec)

   Total (in+out): 14 tokens at 16.94 tokens/sec


⏱️ Translation: 19 tokens processed in 1.32s (14.44 tokens/sec)

   Total (in+out): 32 tokens at 24.33 tokens/sec


⏱️ Translation: 450 tokens processed in 64.89s (6.93 tokens/sec)

   Total (in+out): 1474 tokens at 22.71 tokens/sec


⏱️ Translation: 4 tokens processed in 0.41s (9.65 tokens/sec)

   Total (in+out): 11 tokens at 26.54 tokens/sec


⏱️ Translation: 3 tokens processed in 0.27s (11.20 tokens/sec)

   Total (in+out): 7 tokens at 26.13 tokens/sec


⏱️ Translation: 6 tokens processed in 0.36s (16.63 tokens/sec)

   Total (in+o

In [23]:
for original, translated in zip(grouped_chunks['The_Plan_of_the_Giza_Pyramids.pdf'], translated_jsons):
    print("Original Chunk:")
    print(original)
    print("\nTranslated Chunk:")
    print(translated)
    print("-" * 20)

Original Chunk:
{'source': 'The_Plan_of_the_Giza_Pyramids.pdf', 'pages': [1], 'chunk_number': 0, 'text': 'The Plan of the Giza Pyramids 1', 'token_count': 9, 'element_types': ['header', 'uncategorizedtext']}

Translated Chunk:
{'source': 'خطة الأهرامات في غيزا.pdf', 'pages': [1], 'chunk_number': 0, 'text': 'خطة أهرامات غزة 1', 'token_count': 9, 'element_types': ['الرأس', 'النص غير المصنف']}
--------------------
Original Chunk:
{'source': 'The_Plan_of_the_Giza_Pyramids.pdf', 'pages': [1], 'chunk_number': 1, 'text': "The Plan of the Giza Pyramids by John A.R. Legon Revision Date: 22-11-2019 My initial findings on the Giza Site Plan of Three Pyramids were first briefly summarized in a pamphlet published by the Archaeology Society of Staten Island in 1979.1 Subsequently, I described the most significant features of the integrated site plan in several articles in the journals Discussions in Egyptology2 and Göttinger Miszellen.3 Further research has shown that many more factors must be taken

#### **Translating entire file now**

In [ ]:
translator.translate_file(file_path='/content/Stats.docx', target_language='ar', source_language='en')

⏱️ Translation: 5 tokens processed in 0.67s (7.50 tokens/sec)
   Total (in+out): 9 tokens at 13.49 tokens/sec
⏱️ Translation: 132 tokens processed in 10.60s (12.46 tokens/sec)
   Total (in+out): 278 tokens at 26.24 tokens/sec
⏱️ Translation: 7 tokens processed in 0.57s (12.20 tokens/sec)
   Total (in+out): 17 tokens at 29.64 tokens/sec
⏱️ Translation: 62 tokens processed in 3.38s (18.35 tokens/sec)
   Total (in+out): 119 tokens at 35.21 tokens/sec
⏱️ Translation: 249 tokens processed in 16.90s (14.73 tokens/sec)
   Total (in+out): 520 tokens at 30.77 tokens/sec
⏱️ Translation: 8 tokens processed in 0.97s (8.23 tokens/sec)
   Total (in+out): 19 tokens at 19.55 tokens/sec
⏱️ Translation: 161 tokens processed in 12.06s (13.35 tokens/sec)
   Total (in+out): 362 tokens at 30.02 tokens/sec
⏱️ Translation: 86 tokens processed in 6.56s (13.11 tokens/sec)
   Total (in+out): 206 tokens at 31.40 tokens/sec
⏱️ Translation: 107 tokens processed in 10.41s (10.28 tokens/sec)
   Total (in+out): 288 to

'/content/translated/Stats_ar.docx'

<div style="text-align: center;">
  <img src="https://github.com/UzairNaeem3/DrX_EnigmaticResearch/raw/master/images/Screenshot_8.png" style="display: inline-block" />
</div>


**Not perfect, but better**

<div style="text-align: center;">
  <img src="https://github.com/UzairNaeem3/DrX_EnigmaticResearch/raw/master/images/Screenshot_9.png" style="display: inline-block" />
</div>


### **Translating a PDF file now**

In [ ]:
translator.translate_file(file_path='/content/The_Plan_of_the_Giza_Pyramids.pdf', target_language='ar', source_language='en')

Starting page-by-page translation of PDF: /content/The_Plan_of_the_Giza_Pyramids.pdf
PDF has 16 pages
Processing page 1 of 16
⏱️ Translation: 643 tokens processed in 26.64s (24.14 tokens/sec)

   Total (in+out): 1091 tokens at 40.96 tokens/sec


Saved translated page 1 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_1.txt
Processing page 2 of 16
⏱️ Translation: 438 tokens processed in 18.10s (24.20 tokens/sec)

   Total (in+out): 737 tokens at 40.72 tokens/sec


Saved translated page 2 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_2.txt
Processing page 3 of 16
⏱️ Translation: 697 tokens processed in 37.14s (18.77 tokens/sec)

   Total (in+out): 1267 tokens at 34.11 tokens/sec


Saved translated page 3 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_3.txt
Processing page 4 of 16
⏱️ Translation: 803 tokens processed in 33.95s (23.65 tokens/sec)

   Total (in+out): 1240 tokens at 36.52 tokens/sec


Saved translated page 4 to /cont

Token indices sequence length is longer than the specified maximum sequence length for this model (1202 > 1024). Running this sequence through the model will result in indexing errors


⏱️ Translation: 980 tokens processed in 111.31s (8.80 tokens/sec)

   Total (in+out): 2182 tokens at 19.60 tokens/sec


Saved translated page 5 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_5.txt
Processing page 6 of 16
⏱️ Translation: 819 tokens processed in 74.48s (11.00 tokens/sec)

   Total (in+out): 2137 tokens at 28.69 tokens/sec


Saved translated page 6 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_6.txt
Processing page 7 of 16
⏱️ Translation: 463 tokens processed in 19.44s (23.82 tokens/sec)

   Total (in+out): 790 tokens at 40.64 tokens/sec


Saved translated page 7 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_7.txt
Processing page 8 of 16
⏱️ Translation: 333 tokens processed in 55.91s (5.96 tokens/sec)

   Total (in+out): 614 tokens at 10.98 tokens/sec


Saved translated page 8 to /content/translated/The_Plan_of_the_Giza_Pyramids_ar_pages/page_8.txt
Processing page 9 of 16
⏱️ Translation: 477 tokens processed in

'/content/translated/The_Plan_of_the_Giza_Pyramids_ar.txt'



> It does translate the PDF, but after translation, it wasn't converting back into a PDF correctly. So, I saved the translated text as a .txt file instead of a PDF. However, when dealing with complex PDFs that contain tables, the results were still not satisfactory

